## Imports from multilogger module

In [ ]:
import multilogger
from multilogger import HelloMultiLogger, MLCallback

## Simple Example: Basic structure of generic callback routine receiving all events:

In [ ]:
def callback(event, payload):
    if event == "ML_BEGIN_LOG":
        pass
    elif event == "ML_LOG_DATA":
        pass
    elif event == "ML_END_LOG":
        pass
    else:
        print("Received unknown event: %r"%event)

## Simple Example #2: A callback displaying the event name and payload

In [ ]:
def my_callback(event, payload):
    print("Got event:", event)
    if event == "ML_BEGIN_LOG":
        print(*payload)
    elif event == "ML_LOG_DATA":
        print(*payload)
    elif event == "ML_END_LOG":
        pass
    else:
        print("Received unknown event: %r"%event)

## Simple CSV Writer: Using callbacks to open a file, log data received, and close file when done

In [ ]:
def csv_writer(filename):
    
    if os.path.exists(filename):
        raise FileExistsError(filename)
    
    f = None
    pvars = None
    
    def log(data):
        nonlocal f
        f.write(",".join(map(str, data))); f.write("\n")
    
    def csv_writer_cb(event, payload):
        nonlocal f, pvars
        
        if event == "ML_BEGIN_LOG":
            f = open(filename, 'w')
            
            # create pvars
            pvars = []
            for tag, vl in payload:
                for v in vl:
                    pvars.append(tag + "." + v)
            log(pvars)
        
        elif event == "ML_LOG_DATA":
            line = []
            for tag, vl in payload:
                for v in vl:
                    line.append(v)
            log(line)
        
        elif event == "ML_END_LOG":
            f.close()
        
        else:
            raise ValueError("Unknown event: %r"%event)
    
    return csv_writer_cb

## Simple CSV Writer #2: Class based callback handler.

Instead of using a callback function to process all events, the MLCallback class provides methods to shutter specific events to specific functions

In [ ]:
class CSVWriter(MLCallback):
    def __init__(self, filename):
        super().__init__()
        if os.path.exists(filename):
            raise FileExistsError(filename)
        self._filename = filename
        self._f = None
        self._pvars = None
        
    def log_data(self, data):
        if self._f is None:
            raise ValueError("File not initialized")
        self._f.write(",".join(map(str,data)))
        self._f.write("\n")
        
    def onBeginLog(self, payload):
        
        # check file existance again when starting, just in case
        if os.path.exists(self._filename):
            raise FileExistsError(self._filename)
        
        self._f = open(self._filename, 'w')
        
        self._pvars = []
        for tag, vl in payload:
            for v in vl:
                self._pvars.append(tag + "." + v)
                
        self.log_data(self._pvars)
        
    def onDataReceived(self, payload):
        line = []
        for tag, vl in payload:
            for v in vl:
                line.append(v)
                
        self.log_data(line)
        
    def onEndLog(self, payload):
        self._f.close()
        self._f = None

## Example Usage of HelloMultiLogger

In [ ]:
import time

# Choose all variables to log. All reactors will log the same variables.
# If you need to find out what variables are loggable, call multilogger.get_loggable_vars()

vars_i_care_about = [
    'agitation.pv',
    'temperature.pv'
]

# Create logger 
ml = HelloMultiLogger(vars_i_care_about, 3)

# Add each bioreactor spec as a separate logger
ml.add_logger("192.168.1.12", "R&D1", 1)
ml.add_logger("192.168.1.16", "R&D2", 1)

# Add some simple callbacks (see above)
ml.add_callback(my_callback)
ml.add_callback(csv_writer("test_multilogger.csv"))

# These two are equivalent
ml.add_callback(CSVWriter("test_multilogger2.csv"))
ml.add_callback(CSVWriter("test_multilogger3.csv").onEvent)

ml.start()

In [ ]:
ml.stop()

## The multilogger refuses to overwrite existing CSV Files

Clean up after this example is done so we can rerun it again. Don't do this if you're collecting real data!

In [ ]:
os.remove("test_multilogger.csv")
os.remove("test_multilogger2.csv")
os.remove("test_multilogger3.csv")